**Lab2-DL: E1-emosjoner-bygging.ipynb** (ELMED219) | Prioritet: 3 (valgfri)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arvidl/ELMED219-2026/blob/main/Lab2-DL/notebooks/E1-emosjoner-bygging.ipynb)

# 😊 E1: CNN-klassifikasjon av ansiktsutrykk - Del 1 (Bygging)

Denne notebook demonstrerer hvordan **Konvolusjonelle Nevrale Nettverk (CNN)** kan brukes til å klassifisere ansiktsutrykk og emosjoner, og sammenligner dette med medisinske anvendelser.

## Mål
- Bygge og trene en CNN for å klassifisere 6 universelle emosjoner
- Forstå hvordan CNN fungerer på ansiktsbilder
- Sammenligne med medisinske anvendelser (depresjon, smerte, nevrologiske tilstander)
- Demonstrere evalueringsmetoder: forvirringsmatrise, CAM/Grad-CAM
- Diskutere etiske aspekter (bias, personvern)
- Illustrere formalismen **y ~ f(X, θ)** i praksis

## Datasett
Vi bruker **FER2013** (Facial Expression Recognition 2013) datasettet med <strike>7</strike> 6 emosjonsklasser:
- **<strike>Anger** (Sinne)</strike> - **denne er tom i FER2013**
- 🤢 **Disgust** (Avsky) 
- 😨 **Fear** (Frykt)
- 😊 **Happy** (Glede)
- 😢 **Sad** (Tristhet)
- 😲 **Surprise** (Overraskelse)
- 😐 **Neutral** (Nøytral)

## Teoretisk Fundament

### Formalismen y ~ f(X, θ)

I maskinlæring kan vi uttrykke emosjonsklassifikasjonsproblemet som:

**y = f(X, θ) + ε**

Hvor:
- **y** = predikert emosjon (0-5)
- **X** = input ansiktsbilde (pikselverdier)
- **θ** = modellparametere (CNN-vekter)
- **f** = ikke-lineær funksjon (CNN-arkitekturen)
- **ε** = feilterm (noise)

Dette er identisk med medisinsk bildeanalyse, bare med forskjellige klasser!

## Sammenligning: Emosjoner vs Medisinske Bilder

| Aspekt | Emosjonsgjenkjenning | Medisinsk Bildeanalyse |
|--------|---------------------|------------------------|
| **Input (X)** | RGB/gråtoner av ansikter | Multiparametrisk MRI, røntgen, CT |
| **Klasser (y)** | 6 universelle emosjoner | Sykdomstilstander, anatomiske strukturer |
| **Kompleksitet** | Mikro-uttrykk, kulturelle forskjeller | Anatomiske strukturer, patologier |
| **Konsekvenser** | Psykologisk vurdering | Frisk - Syk |
| **Datamengde** | Tusener av bilder | Begrenset (privacy, ekspertise) |
| **Ekspertise** | Psykologi, nevrologi | Medisin, radiologi |
| **Bias** | Kulturell, etnisk, kjønnsbias | Demografisk, teknisk bias |

**Felles prinsipper:**
- Begge krever domenekunnskap
- Begge har problemer med ubalanserte klasser  
- Begge trenger robuste modeller
- Begge har etiske implikasjoner

### Men først: 🔧 miljøoppsett - kode skal fungere både lokalt, i Codespaces samt Google Colab

In [1]:
import sys
import subprocess
import os

# Sjekk om vi kjører i Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🚀 Kjører i Google Colab")
else:
    print("💻 Kjører i lokal miljø/Codespaces")

💻 Kjører i lokal miljø/Codespaces


In [2]:
if IN_COLAB:
    # Gå til root-mappen
    os.chdir('/content')
    
    # Sjekk nåværende mappe
    print(f"Nåværende mappe: {os.getcwd()}")
    
    # Sjekk om mappen allerede eksisterer
    if os.path.exists('ELMED219-2026/Lab2-DL'):
        print("✅ ELMED219-2026/Lab2-DL mappen eksisterer allerede!")
        
        # Sjekk innholdet
        print("\n📁 Innhold i ELMED219-2026/Lab2-DL mappen:")
        try:
            result = subprocess.run(['ls', '-la', 'ELMED219-2026/Lab2-DL'], 
                                  capture_output=True, text=True, check=True)
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"❌ Kunne ikke liste innhold: {e}")
        
        # Sjekk om det er en git repository
        if os.path.exists('ELMED219-2026/Lab2-DL/.git'):
            print("\n✅ Dette er en git repository!")
            
            # Gå inn i mappen og oppdater
            os.chdir('ELMED219-2026/Lab2-DL')
            print(f"📁 Byttet til: {os.getcwd()}")
            
            # Prøv å oppdatere repositoryet
            try:
                result = subprocess.run(['git', 'pull'], 
                                      capture_output=True, text=True, check=True)
                print("✅ Repository oppdatert!")
                print(result.stdout)
            except subprocess.CalledProcessError as e:
                print(f"⚠️ Kunne ikke oppdatere repository: {e}")
                print("Men mappen eksisterer og kan brukes!")
        else:
            print("⚠️ Dette ser ikke ut som en git repository")
            
    else:
        print("�� Mappen eksisterer ikke - prøver git clone...")
        try:
            result = subprocess.run(['git', 'clone', 'https://github.com/arvidl/ELMED219-2026.git'], 
                                  capture_output=True, text=True, check=True)
            print("✅ Repository klonet vellykket!")
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"❌ Git clone feilet: {e}")
            print(f"Error output: {e.stderr}")

In [3]:
if IN_COLAB:
    !pip install opencv-python --quiet
    !pip install tqdm --quiet
    !pip install torchsummary --quiet

In [4]:
# Imports og setup med feilhåndtering
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import os
import requests
import zipfile
from PIL import Image
import cv2
from tqdm import tqdm
import warnings
import pickle
from pathlib import Path
warnings.filterwarnings('ignore')

# Sjekk om GPU eller MPS er tilgjengelig og sett enhet (device)
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'🚀 GPU tilgjengelig: {torch.cuda.get_device_name(0)}')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print(' Apple Silicon MPS tilgjengelig')
else:
    device = torch.device('cpu')
    print('💻 Bruker CPU')

print(f'Bruker enhet: {device}')

# Sett random seeds for reproduserbarhet
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

🚀 GPU tilgjengelig: NVIDIA RTX A5000 Laptop GPU
Bruker enhet: cuda


## 1. Data Lasting og Forberedelse

### FER2013 Datasett

**FER2013** (Facial Expression Recognition 2013) er et av de mest brukte datasettene for emosjonsgjenkjenning:

- **Størrelse**: 35,887 bilder
- **Oppløsning**: 48x48 piksler
- **Format**: Gråtoner
- **Klasser**: 6 emosjoner (0-5)  [Oprinnelig 7 universelle emosjoner: "Anger" (sinne) mangler i FER2013]
- **Split**: Training (28,709), PublicTest (3,589), PrivateTest (3,589)

### Datasettstruktur
```
../data/
└── ansiktsuttrykk/
    ├── FER2013/
    │   ├── train/
    │   │   ├── 0_disgust/
    │   │   ├── 1_fear/
    │   │   ├── 2_happy/
    │   │   ├── 3_sad/
    │   │   ├── 4_surprise/
    │   │   └── 5_neutral/
    │   ├── val/
    │   │   ├── 0_disgust/
    │   │   ├── 1_fear/
    │   │   ├── 2_happy/
    │   │   ├── 3_sad/
    │   │   ├── 4_surprise/
    │   │   └── 5_neutral/
    │   └── test/
    │   │   ├── 0_disgust/
    │   │   ├── 1_fear/
    │   │   ├── 2_happy/
    │   │   ├── 3_sad/
    │   │   ├── 4_surprise/
    │   │   └── 5_neutral/
    └── fer2013.csv
```

In [5]:
def hent_fer2013_dataset():
    """
    Hent FER2013 datasettet fra Kaggle (hvis ikke allerede lastet ned)
    """
    print("📥 Henter FER2013 datasett")
    print("=" * 50)
    
    data_dir = Path("../data/ansiktsuttrykk/FER2013")
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # Sjekk om data allerede eksisterer
    existing_images = list(data_dir.glob("**/*.png")) + list(data_dir.glob("**/*.jpg"))
    
    if existing_images:
        print(f"✅ Data allerede eksisterer: {len(existing_images)} bilder")
        return True
    
    # Prøv å laste ned fra Kaggle
    try:
        if IN_COLAB:
            # I Colab, last ned fra Kaggle
            !pip install kaggle --quiet
            # Used to securely store your API key
            from google.colab import userdata

            # Load Kaggle credentials from Colab Secrets
            try:
                os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
                os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
                print("✅ Kaggle credentials loaded from Colab Secrets!")
            except userdata.notebook_secret.NotebookAccessError:
                print("❌ Could not load Kaggle credentials from Colab Secrets.")
                print("Please ensure you have added KAGGLE_USERNAME and KAGGLE_KEY to the Secrets manager (🔑 icon) and enabled 'Notebook access'.")
                os.environ['KAGGLE_USERNAME'] = '' # Clear environment variables if access failed
                os.environ['KAGGLE_KEY'] = ''

            # Re-run the data download cell
            !kaggle datasets download -d msambare/fer2013 -p {data_dir} --unzip
        else:
            # Lokalt, sjekk om Kaggle API er tilgjengelig
            kaggle_path = Path.home() / ".kaggle" / "kaggle.json"
            if kaggle_path.exists():
                print("✅ Kaggle API credentials funnet!")
                print("🔄 Prøver automatisk nedlasting...")
                
                # Last ned datasettet
                import subprocess
                result = subprocess.run([
                    "kaggle", "datasets", "download", 
                    "-d", "msambare/fer2013", 
                    "-p", str(data_dir), "--unzip"
                ], capture_output=True, text=True)
                
                if result.returncode == 0:
                    print("✅ Datasett lastet ned!")
                else:
                    print(f"❌ Nedlasting feilet: {result.stderr}")
                    return False
            else:
                print("⚠️ Kaggle API credentials ikke funnet!")
                print("Gå til: https://www.kaggle.com/datasets/msambare/fer2013")
                print(f"Last ned og ekstraher til: {data_dir}")
                return False
        
        # Sjekk om nedlasting var vellykket
        downloaded_images = list(data_dir.glob("**/*.png")) + list(data_dir.glob("**/*.jpg"))
        if downloaded_images:
            print(f"✅ Nedlasting vellykket: {len(downloaded_images)} bilder")
            return True
        else:
            print("❌ Ingen bilder funnet etter nedlasting!")
            return False
            
    except Exception as e:
        print(f"❌ Feil under nedlasting: {e}")
        return False



In [6]:
%%time
# Hent data (hvis ikke allerede lastet ned)
success = hent_fer2013_dataset()

📥 Henter FER2013 datasett
✅ Kaggle API credentials funnet!
🔄 Prøver automatisk nedlasting...
✅ Datasett lastet ned!
✅ Nedlasting vellykket: 35887 bilder
CPU times: user 216 ms, sys: 26.1 ms, total: 242 ms
Wall time: 5.53 s


In [7]:
def organiser_fer2013_dataset():
    """
    Organiser FER2013 datasettet i train/val/test mapper
    """
    print("🔧 Organiserer FER2013 datasett")
    print("=" * 50)
    
    data_dir = Path("../data/ansiktsuttrykk/FER2013")
    
    # Emosjonsklasser
    emotion_classes = {
        0: 'disgust', 
        1: 'fear',
        2: 'happy',
        3: 'sad',
        4: 'surprise',
        5: 'neutral'
    }
    
    # Opprett undermapper for alle splits og emosjonsklasser
    for split in ['train', 'val', 'test']:
        for emotion in emotion_classes.values():
            (data_dir / split / emotion).mkdir(parents=True, exist_ok=True)
    
    # Sjekk om data allerede er organisert
    val_images = list((data_dir / "val").glob("**/*.png")) + list((data_dir / "val").glob("**/*.jpg"))
    
    if val_images:
        print("✅ Data allerede organisert!")
        return True
    
    # Hvis val-mappen er tom, flytt bilder fra train
    print("🔄 Val-mappen er tom - flytter bilder fra train...")
    
    total_moved = 0
    
    # Gå gjennom hver emosjonsklasse
    for emotion in emotion_classes.values():
        train_emotion_dir = data_dir / "train" / emotion
        val_emotion_dir = data_dir / "val" / emotion
        
        # Finn alle bilder i denne emosjonsklassen
        emotion_images = list(train_emotion_dir.glob("*.png")) + list(train_emotion_dir.glob("*.jpg"))
        
        if not emotion_images:
            print(f"  ⚠️ {emotion}: Ingen bilder funnet i train")
            continue
            
        # Flytt 15% av bildene til val
        val_count = int(len(emotion_images) * 0.15)
        
        if val_count > 0:
            print(f"  {emotion}: {len(emotion_images)} bilder → flytter {val_count} til val")
            
            # Flytt bildene
            moved_count = 0
            for image_path in emotion_images:
                if moved_count >= val_count:
                    break
                    
                # Opprett ny sti i val-mappen
                val_path = val_emotion_dir / image_path.name
                
                # Flytt bildet
                try:
                    image_path.rename(val_path)
                    moved_count += 1
                    total_moved += 1
                except Exception as e:
                    print(f"⚠️ Kunne ikke flytte {image_path.name}: {e}")
        else:
            print(f"  {emotion}: {len(emotion_images)} bilder → ingen flyttet (for få bilder)")
    
    print(f"✅ Flyttet totalt {total_moved} bilder til val!")
    
    # Vis statistikk
    print("\n�� Ny organisering:")
    for split in ['train', 'val', 'test']:
        split_images = list((data_dir / split).glob("**/*.png")) + list((data_dir / split).glob("**/*.jpg"))
        print(f"  {split}: {len(split_images)} bilder")
        
        # Vis per emosjonsklasse
        for emotion in emotion_classes.values():
            emotion_images = list((data_dir / split / emotion).glob("*.png")) + list((data_dir / split / emotion).glob("*.jpg"))
            if emotion_images:
                print(f"    {emotion}: {len(emotion_images)} bilder")
    
    return True

In [8]:
# Organiser data (hvis ikke allerede organisert)
if success:
    organiser_fer2013_dataset()

🔧 Organiserer FER2013 datasett
🔄 Val-mappen er tom - flytter bilder fra train...
  disgust: 436 bilder → flytter 65 til val
  fear: 4097 bilder → flytter 614 til val
  happy: 7215 bilder → flytter 1082 til val
  sad: 4830 bilder → flytter 724 til val
  surprise: 3171 bilder → flytter 475 til val
  neutral: 4965 bilder → flytter 744 til val
✅ Flyttet totalt 3704 bilder til val!

�� Ny organisering:
  train: 25005 bilder
    disgust: 371 bilder
    fear: 3483 bilder
    happy: 6133 bilder
    sad: 4106 bilder
    surprise: 2696 bilder
    neutral: 4221 bilder
  val: 3704 bilder
    disgust: 65 bilder
    fear: 614 bilder
    happy: 1082 bilder
    sad: 724 bilder
    surprise: 475 bilder
    neutral: 744 bilder
  test: 7178 bilder
    disgust: 111 bilder
    fear: 1024 bilder
    happy: 1774 bilder
    sad: 1247 bilder
    surprise: 831 bilder
    neutral: 1233 bilder


In [9]:
def check_fer2013_data():
    """Sjekk om FER2013 data eksisterer og er organisert"""
    data_dir = Path("../data/ansiktsuttrykk/FER2013")
    
    # Sjekk om bilder eksisterer
    existing_images = list(data_dir.glob("**/*.jpg")) + list(data_dir.glob("**/*.png"))
    
    if existing_images:
        print(f"✅ FER2013 data eksisterer: {len(existing_images)} bilder")
        
        # Tell bilder per split og emosjon
        splits = ['train', 'val', 'test']
        emotions = ['anger', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        
        for split in splits:
            print(f"\n{split.upper()}:")
            for emotion in emotions:
                count = len(list((data_dir / split / emotion).glob("*.jpg")) + 
                           list((data_dir / split / emotion).glob("*.png")))
                if count > 0:
                    print(f"  {emotion}: {count} bilder")
        
        return True
    else:
        print("❌ Ingen FER2013 data funnet")
        return False

# Kjør sjekk
data_ready = check_fer2013_data()

✅ FER2013 data eksisterer: 35887 bilder

TRAIN:
  disgust: 371 bilder
  fear: 3483 bilder
  happy: 6133 bilder
  sad: 4106 bilder
  surprise: 2696 bilder
  neutral: 4221 bilder

VAL:
  disgust: 65 bilder
  fear: 614 bilder
  happy: 1082 bilder
  sad: 724 bilder
  surprise: 475 bilder
  neutral: 744 bilder

TEST:
  disgust: 111 bilder
  fear: 1024 bilder
  happy: 1774 bilder
  sad: 1247 bilder
  surprise: 831 bilder
  neutral: 1233 bilder


In [10]:
class EmotionDataset(Dataset):
    """Custom dataset for emotion classification"""
    
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('L')  # Gråtoner
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [11]:
def load_emotion_data(data_dir, test_size=0.2, val_size=0.2):
    """Last og splitt emosjonsdatasett"""
    
    # Emosjonsklasser
    emotion_classes = {
        0: 'disgust', 
        1: 'fear',
        2: 'happy',
        3: 'sad',
        4: 'surprise',
        5: 'neutral'
    }
    
    # Samle alle bilde-sti og etiketter
    image_paths = []
    labels = []
    
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(data_dir, split)
        if os.path.exists(split_dir):
            for emotion_name in emotion_classes.values():
                emotion_dir = os.path.join(split_dir, emotion_name)
                if os.path.exists(emotion_dir):
                    for filename in os.listdir(emotion_dir):
                        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                            image_paths.append(os.path.join(emotion_dir, filename))
                            # Finn emosjonsindeks
                            emotion_idx = [k for k, v in emotion_classes.items() if v == emotion_name][0]
                            labels.append(emotion_idx)
    
    print(f"Totalt bilder funnet: {len(image_paths)}")
    print(f"Emosjoner: {list(emotion_classes.values())}")
    print(f"Bilder per emosjon: {np.bincount(labels)}")
    
    # Splitt data hvis nødvendig
    if len(set(labels)) > 1:  # Sjekk om vi har flere klasser
        X_train, X_temp, y_train, y_temp = train_test_split(
            image_paths, labels, test_size=test_size + val_size, 
            random_state=42, stratify=labels
        )
        
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=test_size/(test_size + val_size),
            random_state=42, stratify=y_temp
        )
    else:
        # Fallback hvis vi bare har én klasse
        X_train, X_val, X_test = image_paths, image_paths, image_paths
        y_train, y_val, y_test = labels, labels, labels
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), list(emotion_classes.values())

In [12]:
# Data transformasjoner
train_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normaliser gråtoner
])

val_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

print("✅ Data transformasjoner definert!")

✅ Data transformasjoner definert!


## 2. CNN Modell for Emosjonsgjenkjenning

In [13]:
class EmotionNet(nn.Module):
    """CNN for emosjonsklassifikasjon"""
    
    emotion_classes = {
        0: 'disgust', 
        1: 'fear',
        2: 'happy',
        3: 'sad',
        4: 'surprise',
        5: 'neutral'
    }
    
    def __init__(self, num_classes=len(emotion_classes), dropout_rate=0.5):
        super(EmotionNet, self).__init__()
        
        # Feature extraction layers
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Block 2  
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            # Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
        )
        
        # Adaptive pooling for ulike input-størrelser
        self.adaptive_pool = nn.AdaptiveAvgPool2d((3, 3))
        
        # Klassifikator
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 3 * 3, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x)
        x = self.classifier(x)
        return x

### Forklaring av EmotionNet arkitekturen

**EmotionNet** er spesielt designet for emosjonsgjenkjenning med følgende egenskaper:

#### **1. Input Layer**
- **1 kanal**: Gråtoner (FER2013 er gråtoner)
- **48x48 piksler**: Standard FER2013 oppløsning

#### **2. Feature Extraction (4 blokker)**
```python
# Block 1: Grunnleggende kanter og teksturer
Conv2d(1, 32, 3x3) → BatchNorm → ReLU → MaxPool(2x2) → Dropout2d(0.25)

# Block 2: Mer komplekse mønstre
Conv2d(32, 64, 3x3) → BatchNorm → ReLU → MaxPool(2x2) → Dropout2d(0.25)

# Block 3: Høyere nivå features
Conv2d(64, 128, 3x3) → BatchNorm → ReLU → MaxPool(2x2) → Dropout2d(0.25)

# Block 4: Komplekse emosjonelle features
Conv2d(128, 256, 3x3) → BatchNorm → ReLU → MaxPool(2x2) → Dropout2d(0.25)
```

#### **3. Regularisering**
- **BatchNorm2d**: Stabiliserer trening
- **Dropout2d**: Forhindrer overfitting
- **Dropout**: I fully-connected layers

#### **4. Klassifikator**
- **AdaptiveAvgPool2d(3x3)**: Reduserer til 3x3 spatial dimensjoner
- **3 FC layers**: 256×9 → 512 → 256 → 6
- **6 output klasser**: 6 emosjoner

## 3. Treningsfunksjoner

In [14]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Tren model for én epoke"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    """Valider model for én epoke"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    
    return epoch_loss, epoch_acc

In [15]:
def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001):
    """Tren den fullstendige modellen"""
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    best_val_acc = 0.0
    patience_counter = 0
    
    print("Starter trening...")
    print("=" * 50)
    
    for epoch in range(num_epochs):
        print(f"\nEpoke {epoch+1}/{num_epochs}")
        print("-" * 30)
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validate
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Store metrics
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), './modeller/best_emotion_model.pth')
        else:
            patience_counter += 1
            
        if patience_counter >= 10:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses, 
        'train_accs': train_accs,
        'val_accs': val_accs,
        'best_val_acc': best_val_acc
    }

## 4. Hovedkjøring - Del 1

In [16]:
# Hovedkjøring med automatisk oppsett
def main_part1():
    """Hovedkjøring del 1: Data lasting og modell oppsett"""
    
    print(" Ansiktsutrykk-klassifikasjon med CNN - Del 1")
    print("=" * 60)
    
    # Sett opp data-mapper
    data_dir = "../data/ansiktsuttrykk/FER2013"
    
    # Sjekk om data eksisterer
    if not os.path.exists(data_dir) or len(os.listdir(data_dir)) == 0:
        print("❌ Data directory not found or empty!")
        print("\n🔄 Prøver automatisk oppsett...")
        
        # Prøv automatisk nedlasting
        success = download_fer2013_dataset()
        
        if not success:
            print("\n❌ Automatisk oppsett feilet!")
            print("\n📋 Manuell nedlasting:")
            print("1. Gå til: https://www.kaggle.com/datasets/msambare/fer2013")
            print("2. Last ned 'fer2013.zip'")
            print("3. Ekstraher til '../data/ansiktsuttrykk/' mappen")
            return None, None, None, None, None
    else:
        print("✅ Data-katalog funnet!")
    
    # Last data
    print("\n Laster data...")
    (X_train, y_train), (X_val, y_val), (X_test, y_test), class_names = load_emotion_data(data_dir)
    
    # Etabler datasett
    print("\n🔧 Etablerer trening-, validering- og test-datasett...")
    train_dataset = EmotionDataset(X_train, y_train, train_transform)
    val_dataset = EmotionDataset(X_val, y_val, val_transform)
    test_dataset = EmotionDataset(X_test, y_test, val_transform)
    
    # Etabler data-laster 
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
    
    print(f"✅ Lasting av data vellykket!")
    print(f"Trening: {len(train_dataset)} bilder")
    print(f"Validering: {len(val_dataset)} bilder") 
    print(f"Test: {len(test_dataset)} bilder")
    
    # Bygg modell
    print("\n��️ Bygger modell...")
    model = EmotionNet(num_classes=len(class_names)).to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    return model, train_loader, val_loader, test_loader, class_names



In [17]:
# Kjør hovedkjøring
model, train_loader, val_loader, test_loader, class_names = main_part1()

 Ansiktsutrykk-klassifikasjon med CNN - Del 1
✅ Data-katalog funnet!

 Laster data...
Totalt bilder funnet: 30934
Emosjoner: ['disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
Bilder per emosjon: [ 547 5121 8989 6077 4002 6198]

🔧 Etablerer trening-, validering- og test-datasett...
✅ Lasting av data vellykket!
Trening: 18560 bilder
Validering: 6187 bilder
Test: 6187 bilder

��️ Bygger modell...
Model parameters: 1,701,830


In [18]:
# Vis modellstruktur
print("\n📊 Modell Sammendrag:")
print("=" * 50)

# Test modellen med dummy input først
print("🧪 Tester modell med dummy input...")
try:
    dummy_input = torch.randn(1, 1, 48, 48).to(device)
    with torch.no_grad():
        output = model(dummy_input)
    print(f"✅ Modell test vellykket!")
    print(f"Input shape: {dummy_input.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Output device: {output.device}")
except Exception as e:
    print(f"❌ Modell test feilet: {e}")

# Vis modellstruktur manuelt
print(f"\n📋 Modell Detaljer:")
print(f"Modell: {model}")
print(f"Enhet: {device}")
print(f"Input størrelse: (batch_size, 1, 48, 48)")
print(f"Output størrelse: (batch_size, {len(class_names)})")

# Prøv torchsummary hvis modellen fungerer
try:
    from torchsummary import summary
    # Sørg for at modellen er på CPU for torchsummary
    model_cpu = model.cpu()
    summary(model_cpu, input_size=(1, 48, 48))  # (kanaler, høyde, bredde)
    # Flytt modellen tilbake til riktig enhet
    model = model.to(device)
except ImportError:
    print("\n⚠️ torchsummary ikke tilgjengelig - installer med: pip install torchsummary")
except Exception as e:
    print(f"\n⚠️ torchsummary feilet: {e}")
    print("Bruker manuell modellvisning i stedet")


📊 Modell Sammendrag:
🧪 Tester modell med dummy input...
✅ Modell test vellykket!
Input shape: torch.Size([1, 1, 48, 48])
Output shape: torch.Size([1, 6])
Output device: cuda:0

📋 Modell Detaljer:
Modell: EmotionNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentu

## 5. Oppsummering av Del 1

**Del 1** av notebooken har nå:

✅ **Miljøoppsett** - Fungerer på Colab, Codespaces og lokalt<br>
✅ **Device detection** - Automatisk GPU/MPS/CPU valg<br>
✅ **Data lasting** - FER2013 datasett med 7 emosjonsklasser<br>
✅ **Modell definisjon** - EmotionNet CNN arkitektur<br>
✅ **Treningsfunksjoner** - Komplette trenings- og valideringsfunksjoner<br>
✅ **Hovedkjøring** - Automatisk oppsett og modell initialisering<br>

**Neste steg** (Del 2) vil inkludere:
- Visuell inspeksjon av data
- Modell trening
- Evaluering og visualisering
- CAM/Grad-CAM for forklarbar AI
- Medisinske anvendelser og etiske diskusjoner